# Gauge Observables

--------------------------------------------------------------------
### TODO:
* [x] Generate multiple chain lengths and deal with loading in from multiple `samples_history` files.
* [x] Implement the same logic for `observables` as for `samples_history`.
* [x] Modify remainder of code below to deal with case where `samples` and `observables` are dictionaries with keys specifying the length of the MCMC chain.
* [x] Re-run the cells below for the remainder of `HMC` directory to get ESS values for comparing against ESS from L2HMC.
* [x] Try training sampler for >> 1000 steps and running the trained sampler for a variety of different chain lengths to see what the integrated autocorrelation time approaches as  $N_{steps} \longrightarrow \infty$.
--------------------------------------------------------------------

## Imports

In [ ]:
import os
import sys
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

from pandas.plotting import autocorrelation_plot
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
COLORS = 5 * ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
MARKERS = 5 * ['o', 's', 'x', 'v', 'h', '^', 'p', '<', 'd', '>', 'P', 'D']
LINESTYLES = ['-', '--', ':', '-.', '-', '--', ':', '-.', '-', '--']

In [ ]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [ ]:
from lattice.lattice import GaugeLattice, u1_plaq_exact
#from l2hmc_eager import gauge_dynamics_eager as gde
from gauge_model import GaugeModel, save_params_to_pkl_file

import utils.gauge_model_helpers as helpers
from utils.autocorr import *
from utils.gauge_observables import *
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib inline

In [ ]:
def check_else_make_dir(d):
    if not os.path.isdir(d):
        print(f"Making directory: {d}")
        os.makedirs(d)

In [ ]:
def _plot_individual_observables(figs_dir, observables, top_charges_autocorr):
    multiple_lines_figs_axes = make_multiple_lines_plots(
        figs_dir,
        params['beta_final'],
        observables,
        top_charges_autocorr,
        legend=False
    )
    return multiple_lines_figs_axes

def _plot_individual_acf_iat(acf_arr, iat_arr, ess_arr, figs_dir):
    out_file = os.path.join(
        figs_dir, 
        'integrated_autocorrelation_time_plot.pdf'
    )
    kwargs = {
        'x_label': 'Lag',
        'y_label': 'Autocorrelation (top. charge)',
        'legend': True,
        'out_file': out_file
    }
    fig, ax = plot_autocorr_with_iat(acf_arr, iat_arr, ess_arr, **kwargs)
    
    return fig, ax

## Calculate and plot observables...

### Calculate observables for samples generated during **_training_**:
- Every $\approx 500$ steps or so during training procedure, we run the sampler at $\beta \equiv \beta_{\mathrm{final}}$. 
- By calculating observables (``total action``, ``average plaquette``, and ``topological charge``) for these samples and looking at the ``thermalization time``, we can get an idea of how well the sampler is performing.
- We expect that as the training procedure continues, the ``thermalization time`` should decrease as the sampler improves.

In [ ]:
# 53.3s
plt.close('all')
log_dir = os.path.join('..', '..', 'gauge_logs_graph', 'run_227')
train_observables_dicts = calc_observables(log_dir,
                                           observables_dicts=None,
                                           training=True,
                                           frac=None)

####  **Update train_observables_dicts:**  
 * If `observables_dicts` argument of `calc_training_observables` is not `None`, only calculate observables that haven't been previously calculated.  

In [ ]:
train_observables_dicts = calc_observables(
    log_dir, 
    observables_dicts=train_observables_dicts,
    training=True,
)

#### **Plot observables for samples generated during training:**
- In addition, for each batch of samples generated during, plot the topological charge history of each individual chain in the batch.

In [ ]:
figs_axes = plot_observables(log_dir, train_observables_dicts, training=True)

In [ ]:
plot_top_charges(log_dir, train_observables_dicts[2], training=True)

In [ ]:
plt.close('all')
plot_top_charges_counts(log_dir, 
                        train_observables_dicts[2], 
                        training=True)

In [ ]:
train_charges_dict = train_observables_dicts[2]
params, _, _, _, figs_dir_dict = find_samples(log_dir, training=True)
title_str_key = 'training'
count_dict = {}
idx = 0
for key, val in train_charges_dict.items():
    step, beta = key
    counts = Counter(list(val.flatten()))
    count_dict[key] = counts
    fig, ax = plt.subplots()
    _ = ax.plot(list(counts.keys()), list(counts.values()), 
                color=COLORS[idx], marker=MARKERS[idx], ls='')
                #fillstyle='none')#, label=f'{key} training steps')
    idx += 1
    _ = ax.set_xlabel('Topological charge', fontsize=14)
    _ = ax.set_ylabel('Number of occurrences', fontsize=14)
    title_str = (r"$\beta = $"
                 + f"{beta}, "
                 + f"{step} {title_str_key} steps; "
                 + f"total across {params['num_samples']} samples")
    _ = ax.set_title(title_str, fontsize=14)
    out_dir = os.path.join(
        figs_dir_dict[key], 'topological_charges_counts'
    )
    check_else_make_dir(out_dir)
    out_file = os.path.join(
        out_dir,
        f'topological_charge_counts_total_{step}_steps_beta_{beta}.pdf'
    )
    #if not os.path.isfile(out_file):
    print(f"Saving figure to {out_file}.")
    _ = fig.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
plt.close('all')

### Calculate observables for samples generated **_after_** training.
 - Again, samples are generated at $\beta = \beta_{\mathrm{final}}$.
 - In contrast to the samples generated **_during_** training (which are all ran for $\sim 100$ steps), we now look at generating longer chains (i.e. longer runs).

In [ ]:
plt.close('all')
log_dir = os.path.join('..', '..', 'gauge_logs_graph', 'run_227')
observables_dicts = calc_observables(log_dir, 
                                     observables_dicts=None, 
                                     training=False,
                                     frac=4)

In [ ]:
observables_dicts = calc_observables(log_dir, 
                                     observables_dicts=observables_dicts, 
                                     training=False,
                                     frac=4)

In [ ]:
actions_dict = observables_dicts[0]
plaqs_dict = observables_dicts[1]
charges_dict = observables_dicts[2]
suscept_stats_dict = observables_dicts[3]
plaqs_stats_dict = observables_dicts[4]
charges_probs_dict = observables_dicts[5]

In [ ]:
plaqs_arr = plaqs_dict[(20000, 4.0)]
charges_arr = charges_dict[(20000, 4.0)]

In [ ]:
plaqs_arr.shape

In [ ]:
steps = np.arange(plaqs_arr.shape[0])
for idx in range(plaqs_arr.shape[1]):
    fig, ax = plt.subplots()
    _ = ax.plot(steps, plaqs_arr[:, idx], label=f'sample {idx}', marker='', color=COLORS[idx])
    _ = ax.axhline(y=u1_plaq_exact(4.), color='r', ls='--', lw=2.5, label='exact')
    _ = ax.legend(loc='best')
    _ = ax.set_xlabel('Step', fontsize=14)
    _ = ax.set_ylabel('Avg. Plaquette', fontsize=14)
    title_str = (r"$\beta = $"
                 + f"{4.0}, {20000} eval steps")
    _ = ax.set_title(title_str, fontsize=16)
    out_file = os.path.join(log_dir, 'figures', f'avg_plaquette_sample{idx}.pdf')
    plt.savefig(out_file, dpi=400, bbox_inches='tight')
    _ = plt.show()

In [ ]:
steps = np.arange(plaqs_arr.shape[0])
fig, ax = plt.subplots()
ax.plot(steps, plaqs_arr[:, 0], label='sample 0', marker='.', color='C0')
_ = ax.axhline(y=u1_plaq_exact(4.), color='r', ls='--', lw=2.5, label='exact')
ax.legend(loc='best')
ax.set_xlabel('Step')
ax.set_ylabel('Avg. Plaquette')
plt.show()

In [ ]:
plot_observables(log_dir, observables_dicts, training=False)

In [ ]:
plot_top_charges(log_dir, observables_dicts[2], training=False)

In [ ]:
plt.close('all')
plot_top_charges_counts(log_dir, observables_dicts[2], training=False)

In [ ]:
plt.close('all')
COLORS *= 10
MARKERS *= 10
params, _, _, _, figs_dir_dict = find_samples(log_dir)
charges_dict = observables_dicts[2]
title_str_key = 'evaluation'
count_dict = {}
idx = 0
for key, val in charges_dict.items():
    step, beta = key 
    counts = Counter(list(val.flatten()))
    count_dict[key] = counts
    fig, ax = plt.subplots()
    _ = ax.plot(list(counts.keys()), list(counts.values()), 
                color=COLORS[idx], marker=MARKERS[idx], ls='',
                fillstyle='full')#, label=f'{key} training steps')
    idx += 1
    _ = ax.set_xlabel('Topological charge', fontsize=14)
    _ = ax.set_ylabel('Number of occurrences', fontsize=14)
    title_str = (r"$\beta = $"
                 + f"{beta}, "
                 + f"{step} steps; "
                 + f"total across {params['num_samples']} samples")
    _ = ax.set_title(title_str, fontsize=16)
    #out_dir = os.path.join(
    #    figs_dir_dict[key], 'topological_charges_counts'
    #)
    #check_else_make_dir(out_dir)
    out_file = os.path.join(
       figs_dir_dict[key],
        f'topological_charge_counts_total_{step}_steps_beta_{beta}.pdf'
    )
    #if not os.path.isfile(out_file):
    print(f"Saving figure to {out_file}.")
    _ = fig.savefig(out_file, dpi=400, bbox_inches='tight')
    plt.show()
    plt.close('all')

In [ ]:
plt.close('all')

In [ ]:
def update_beta(step, beta_init, beta_final, train_steps):
    """Returns new beta to follow annealing schedule."""
    temp = ((1. / beta_init - 1. / beta_final)
            * (1. - step / float(train_steps))
            + 1. / beta_final)
    new_beta = 1. / temp

    return new_beta

In [ ]:
update_beta(500, 3., 4., 5000)

In [ ]:
def update_beta1(beta, annealing_factor, beta_final):
    new_beta = beta / annealing_factor
    if new_beta < beta_final:
        return new_beta
    return beta_final

In [ ]:
def update_beta2(beta_init, beta_final, train_steps, step):
    """Returns new beta to follow annealing schedule."""
    temp = ((1. / beta_init - 1. / beta_final)
            * (1. - step / float(train_steps))
            + 1. / beta_final)
    new_beta = 1. / temp
    return new_beta

In [ ]:
beta_init = 2.
beta_final = 4.
train_steps = 6931

In [ ]:
betas1 = []
betas2 = []
beta1 = beta_init
for i in range(train_steps):
    beta1 = update_beta1(beta1, 0.9999, beta_final)
    beta2 = update_beta2(beta_init, beta_final, train_steps, i)
    betas1.append(beta1)
    betas2.append(beta2)
betas1 = np.array(betas1)
betas2 = np.array(betas2)

In [ ]:
np.where(betas1 >= 4.)

In [ ]:
x = np.arange(train_steps)
fig, ax = plt.subplots()
ax.plot(x, betas1, label='betas1')
ax.plot(x, betas2, label='betas2')
ax.legend(loc='best')

In [ ]:
x = np.arange(0, 2* np.pi)
y = 1. - np.cos(x)
fig, ax = plt.subplots()
ax.plot(x, y)
plt.show()

# OLD

### Topological susceptibility

$ \chi(\beta, V) \equiv \frac{\left< Q^2 \right>}{V}$

In [ ]:
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats

In [ ]:
actions_dicts, plaqs_dicts, charges_dicts = observables_dicts
charges = charges_dicts[5000]

In [ ]:
test_statistic = lambda x: np.mean(x)

estimate_arr = []
bias_arr = []
stderr_arr = []
conf_interval_arr = []
for idx, sample in enumerate(charges.T):
    sample_squared = sample ** 2
    charge_rs = jackknife_resampling(sample_squared)
    estimate, bias, stderr, conf_interval = jackknife_stats(sample_squared,
                                                            test_statistic,
                                                            0.95)
    print(80 * '-' + '\n')
    print(f"Topological susceptibility statistics for sample {idx}, "
          f"consisting of {charge_rs.shape[0]} L2HMC steps.")
    print(f'estimate: {estimate}')
    print(f'bias: {bias}')
    print(f'stderr: {stderr}')
    print(f'conf_interval: {conf_interval}\n')
    print(80 * '-' + '\n')
    estimate_arr.append(estimate)
    bias_arr.append(bias)
    stderr_arr.append(stderr)
    conf_interval_arr.extend(conf_interval)

In [ ]:
test_statistic = lambda x: np.mean(x)

estimate_arr = []
bias_arr = []
stderr_arr = []
conf_interval_arr = []
for idx, sample in enumerate(charges.T):
    sample_squared = sample ** 2
    charge_rs = jackknife_resampling(sample_squared)
    estimate, bias, stderr, conf_interval = jackknife_stats(sample_squared,
                                                            test_statistic,
                                                            0.95)
    print(80 * '-' + '\n')
    print(f"Topological susceptibility statistics for sample {idx}, "
          f"consisting of {charge_rs.shape[0]} L2HMC steps.")
    print(f'estimate: {estimate}')
    print(f'bias: {bias}')
    print(f'stderr: {stderr}')
    print(f'conf_interval: {conf_interval}\n')
    print(80 * '-' + '\n')
    estimate_arr.append(estimate)
    bias_arr.append(bias)
    stderr_arr.append(stderr)
    conf_interval_arr.extend(conf_interval)

In [ ]:
charges_rs = np.array([jackknife_resampling(sample**2) for sample in charges.T])
charges_rs.shape

In [ ]:
test_statistic = lambda x: np.mean(x))

susceptibility_stats = {
    'estimate': [],
    'bias': [],
    'stderr': [],
    'conf_interval': []
}
for sample in charges_rs:
    estimate, bias, stderr, conf_interval = jackknife_stats(sample,
                                                            test_statistic,
                                                            0.95)
    susceptibility_stats['estimate'].append(estimate)
    susceptibility_stats['bias'].append(bias)
    susceptibility_stats['stderr'].append(bias)
    susceptibility_stats['conf_interval'].append(conf_interval)

In [ ]:
test_statistic = lambda x: (np.mean(x), np.var(x))
estimate, bias, stderr, conf_interval = jackknife_stats(data, 
                                                        test_statistic, 
                                                        0.95)

In [ ]:
charge_squared_mean, charge_squared_err = resampling.jackknife(
    charges, func=lambda x: x**2, func_axis=0
)

In [ ]:
%debug

In [ ]:
squared_mean_a, squared_mean_a_error = resampling.jackknife(a, func=lambda x: x**2)


In [ ]:
suscept = []
errors = []
for sample in charges.T:
    charges_rs = block_resampling(np.array(sample), 2500)
    avg_charge2_rs = []
    for block in charges_rs:
        avg_charge2_rs.append(np.mean(block ** 2))
    error = jackknife_err(y_i=avg_charge2_rs,
                          y_full=sample,
                          num_blocks=2500)
    suscept.append(np.mean(avg_charge2_rs))
    errors.append(error)

In [ ]:
charges = charges_dicts[10000]
print(np.mean(charges**2, axis=0))

In [ ]:
from lattice.lattice import GaugeLattice

In [ ]:
lattice = GaugeLattice(4, 4, 2, 'U1', 5, rand=False, data_format='channels_last')

In [ ]:
lattice_gpu = GaugeLattice(4, 4, 2, 'U1', 5, rand=False, data_format='channels_first')

In [ ]:
lattice.links.shape, lattice_gpu.links.shape

In [ ]:
lattice.samples.shape, lattice_gpu.samples.shape

In [ ]:
for key in lattice.plaquettes_dict.keys():
    print(f'channels_last  {key}: {lattice.plaquettes_dict[key]}')
    print(f'channels_first {key}: {lattice_gpu.plaquettes_dict[key]}')
    print('\n')

In [ ]:
rand_samples = np.random.randn(lattice.num_samples * lattice.num_links)
rand_links = np.random.randn(lattice.num_links)

In [ ]:
s_rand = rand_samples.reshape(lattice.samples.shape)
l_rand = rand_links.reshape(lattice.links.shape)
s_rand.shape, l_rand.shape

In [ ]:
s_rand_gpu = s_rand.transpose(0, 3, 1, 2)
l_rand_gpu = l_rand.transpose(2, 0, 1)
s_rand_gpu.shape, l_rand_gpu.shape

In [ ]:
lattice._total_action(l_rand)

In [ ]:
lattice_gpu._total_action(l_rand_gpu)

In [ ]:
for key in lattice.plaquettes_dict.keys():
    idxs0 = lattice.plaquettes_dict[key]
    idxs1 = lattice_gpu.plaquettes_dict[key]
    plaq_sum0 = [l_rand[idxs0[0]] + l_rand[idxs0[1]] 
                 - l_rand[idxs0[2]] -  l_rand[idxs0[3]]]
    plaq_sum1 = [l_rand_gpu[idxs1[0]] + l_rand_gpu[idxs1[1]] 
                 - l_rand_gpu[idxs1[2]] - l_rand_gpu[idxs1[3]]]
    print(f'plaq_sum (channels last):  {plaq_sum0}')
    print(f'plaq_sum (channels first): {plaq_sum1}')
    print('\n')

In [ ]:
print(lattice.calc_plaq_observables(s_rand, 1.))

In [ ]:
print(lattice_gpu.calc_plaq_observables(s_rand_gpu, 1.))

In [ ]:
lattice._total_action(l_rand)

In [ ]:
lattice_gpu._total_action(l_rand_gpu)

In [ ]:
links_shape = tuple([8] + [8 for _ in range(1)] + [2])
links = np.arange(lattice.num_links).reshape(links_shape)
#links = np.zeros(links_shape, dtype=np.float32)

In [ ]:
plaquettes_arr = np.array(list(lattice.plaquettes_dict.values()))

In [ ]:
print(plaquettes_arr)

In [ ]:
plaquettes_arr.shape

In [ ]:
plaquettes_arr[0, 2:, :]

In [ ]:
pos_plaquettes = plaquettes_arr[:, :2, :]
neg_plaquettes = plaquettes_arr[:, 2:, :]

In [ ]:
action, plaqs, charge = lattice._calc_plaq_observables(links, beta=1.)

In [ ]:
charge

In [ ]:
def project_angle(x):
    return x - 2 * np.pi * tf.math.floor((x + np.pi) / (2 * np.pi))

In [ ]:
pos_plaquettes[0]
np.sum(pos_plaquettes[0], axis=1)
neg_plaquettes[0]

In [ ]:
links(pos_plaquettes[0])

In [ ]:
_charge = np.sum(project_angle(np.sum(links[pos_plaquettes]) - np.sum(links[neg_plaquettes])))

In [ ]:
_charge

In [ ]:
links[0][0]

In [ ]:
links[0, 1]

In [ ]:
links.shape

In [ ]:
links[]

In [ ]:
links

In [ ]:
links = np.array(
    [[i, j, k] for i in range(8) for j in range(8) for k in range(2)]
)
links
links.shape

In [ ]:
lattice.links.shape

In [ ]:
arr = np.array(
    [[a, b, c, d] for a in range(5) for b in range(8) for c in range(8) for d in range(2)]
).reshape(lattice.samples.shape)
arr
arr.shape

In [ ]:
patches = tf.extract_image_patches(images=samples_tensor,
                                   ksizes=[1, 2, 2, 1],
                                   strides=[1, 1, 1, 1],
                                   rates=[1, 1, 1, 1],
                                   padding='VALID')

In [ ]:
patches.shape

In [ ]:
patches[0, 

In [ ]:
  print tf.extract_image_patches(images=images, ksizes=[1, 3, 3, 1], strides=[1, 5, 5, 1], rates=[1, 1, 1, 1], padding='VALID').eval(), '\n\n'


In [ ]:
offsets = np.array(
    [(i, j) for i in range(8) for j in range(8)],  dtype=np.float32
)
offsets
offsets.shape

In [ ]:
samples_tensor = tf.convert_to_tensor(lattice.samples, dtype=tf.float32)
glimpse = tf.image.extract_glimpse(input=samples_tensor, size=(2, 2), offsets=offsets)

In [ ]:
samples_tensor.shape

## Using helpers from: `utils/gauge_observables.py`

In [ ]:
log_dir = (
    '../../gauge_logs_graph/run_48/'
)

calc_observables_generate_plots(log_dir)

In [ ]:
root_dir = ('../../gauge_logs_graph/gauge_logs_by_size/sixteen_by_sixteen/')
log_dirs = [
    root_dir + d for d in os.listdir(root_dir) 
    if os.path.isdir(os.path.join(root_dir, d))
]
log_dirs

In [ ]:
bad_dirs = []
for log_dir in log_dirs:
    try:
        calc_observables_generate_plots(log_dir)
    except:
        bad_dirs.append(log_dir)
        continue

In [ ]:
lattice = GaugeLattice(time_size=8, 
                       space_size=8,
                       dim=2, 
                       beta=8., 
                       link_type='U1',
                       num_samples=5,
                       rand=False)

In [ ]:
lattice.links.shape

## Calculate observables and create plots step by step, dir by dir...

In [ ]:
log_dir = '../../gauge_logs_graph/run_1/'
observables_dicts = calc_observables(log_dir)

In [ ]:
observables_dicts = calc_observables(log_dir, observables_dicts)

In [ ]:
figs_axes = plot_observables(log_dir, observables_dicts)

In [ ]:
plot_top_charges(log_dir, observables_dicts[-1])

In [ ]:
plt.close('all')

In [ ]:
batch_size = 5
forward_mask = tf.cast(
    tf.random_uniform((batch_size,)) > 0.5,
    tf.float32
)
backward_mask = 1. - forward_mask

In [ ]:
lattice = GaugeLattice(8, 8, 2, 'U1', batch_size, rand=False)
position = lattice.samples
position.shape

In [ ]:
position_mask_forward = forward_mask[:, None, None, None] * position

In [ ]:
position_mask_forward.shape

In [ ]:
forward_mask

In [ ]:
backward_mask

### OLD

In [ ]:
log_dir = (
    '../../gauge_logs_graph/run_78/'
)

figs_dir = os.path.join(log_dir, 'figures')
#autocorr_dir = os.path.join(figs_dir, 'autocorrelation_plots')
#check_else_make_dir(autocorr_dir)

#### Calculate observables

In [ ]:
params, samples, observables = calc_observables_from_log_dir(log_dir)
if isinstance(observables, dict):
    actions = {}
    avg_plaquettes = {}
    top_charges = {}
    for key, val in observables.items():
        _actions, _avg_plaquettes, _top_charges = val
        actions[key] = _actions
        avg_plaquettes[key] = _avg_plaquettes
        top_charges[key] = _top_charges
else:
    actions, avg_plaquettes, top_charges = observables

print('\n' + 80 * '-')
for key, val in params.items():
    print(f'{key}: {val}')
print(80 * '-')

#### Calculate autocorr fns, integrated autocorr times (IAT $ = \tau$), and expected sample size (ESS)

In [ ]:
if isinstance(top_charges, dict):
    top_charges_autocorr = {}
    top_charges_autocorr_avg = {}
    acf_dict = {}
    iat_dict = {}
    ess_dict = {}
    for key, val in top_charges.items():
        # Previous (naive) method for calculating the top. charges autocorr fn
        _autocorr, _avg = calc_top_charges_autocorr(val)
        top_charges_autocorr[key] = _autocorr
        top_charges_autocorr_avg[key] = _avg
        
        # New (better) method for calculating the top. charges autocorr fn and 
        # integrated autocorrelation time
        acf_arr, iat_arr = calc_integrated_autocorr_time(val)
        
        ess_arr = []
        for acf in acf_arr:
            ess_arr.append(calc_ESS(acf))
            
        acf_dict[key] = acf_arr
        iat_dict[key] = iat_arr
        ess_dict[key] = ess_arr
        
else:
    output = calc_top_charges_autocorr(top_charges)
    top_charges_autocorr, top_charges_autocorr_avg = output

    # New (better) method for calculating the top. charges autocorr fn and 
    # integrated autocorrelation time:
    acf_arr, iat_arr = calc_integrated_autocorr_time(top_charges)

    ESS_arr = []
    for acf in acf_arr:
        ESS_arr.append(calc_ESS(acf))

# Use naive method for calculating autocorr fn. of invidiual links in samples 
if isinstance(samples, dict):
    samples_autocorr_dict = {}
    samples_autocorr_avg_dict = {}
    for key, val in samples.items():
        samples_autocorr, samples_autocorr_avg = calc_samples_autocorr(val)
        
        samples_autocorr_dict[key] = samples_autocorr
        samples_autocorr_avg_dict[key] = samples_autocorr_avg
else:
    samples_autocorr, samples_autocorr_avg = calc_samples_autocorr(samples)

#### Create plots

In [ ]:
figs_dir_dict = {}
for key in observables.keys():
    new_figs_dir = os.path.join(figs_dir, f'figures_{key}')
    if not os.path.isdir(new_figs_dir):
        print(f'Creating directory: {new_figs_dir}.')
        os.makedirs(new_figs_dir)
        
    figs_dir_dict[key] = new_figs_dir

In [ ]:
for key in observables.keys():
    _figs_dir = figs_dir_dict[key]
    _actions = actions[key]
    _avg_plaquettes = avg_plaquettes[key]
    _top_charges = top_charges[key]
    _top_charges_autocorr = top_charges_autocorr[key]
    _observables = (_actions, _avg_plaquettes, _top_charges)
    
    kwargs = {
        'figs_dir': _figs_dir
    }
    
    figs_axes = make_multiple_lines_plots(
        params['beta_final'], 
        _observables,
        **kwargs
    )
    
    _acf_arr = acf_dict[key]
    _iat_arr = iat_dict[key]
    _ess_arr = ess_dict[key]
    fig, ax = _plot_individual_acf_iat(_acf_arr, _iat_arr, _ess_arr, _figs_dir)

#### Topological charge history

In [ ]:
for key, val in top_charges.items():
    root_dir = figs_dir_dict[key]
    fig_dir = os.path.join(root_dir, 'top_charges_figs')
    check_else_make_dir(fig_dir)
    for idx in range(val.shape[1]):
        fig, ax = plt.subplots()
        _ = ax.plot(val[:, idx], label=f'sample {idx}', 
                    color=COLORS[idx], marker=MARKERS[idx], fillstyle='none', 
                    ls=':', lw=0.75)
        _ = ax.legend(loc='best')
        out_file = os.path.join(fig_dir, 
                                f'top_charge_history_sample_{idx}.pdf')
        print(f'Saving figure to: {out_file}.')
        _ = plt.savefig(out_file, dpi=400, bbox_inches='tight')

#### Histograms for topological charge

In [ ]:
plt.close('all')
for key, val in top_charges.items():
    root_dir = figs_dir_dict[key]
    fig_dir = os.path.join(root_dir, 'top_charges_histograms')
    check_else_make_dir(fig_dir)
    for idx in range(val.shape[1]):
        fig, ax = plt.subplots()
        # the trick is to set up the bins centered on the integers, i.e.
        # -0.5, 0.5, 1,5, 2.5, ... up to max(data) + 1.5. 
        # Then you substract -0.5 to # eliminate the extra bin at the end.
        bins = np.arange(val[:, idx].min(), val[:, idx].max() + 1.5) - 0.5
        _ = ax.hist(
            val[:, idx], 
            bins, 
            color=COLORS[idx], 
            label=f'sample {idx}'
        )
        _ = ax.set_xticks(bins + 0.5)
        #ax.hist(val[:, idx])
        _ = ax.legend(loc='best')
        out_file = os.path.join(
            fig_dir, 
            f'top_charge_history_sample_{idx}_histogram.pdf'
        )
        print(f'Saving figure to: {out_file}')
        _ = plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
plt.close('all')

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def exp_fn(x, a, b, c, d):
    return a * np.exp(-b * (x - c))

In [ ]:
def fit_exp_linear(t, y, C=0):
    y = y - C
    y = np.log(y)
    K, A_log = np.polyfit(t, y, 1)
    A = np.exp(A_log)
    return A, K

In [ ]:
def model_func(t, A, K, C):
    return A * np.exp(K * t) + C

In [ ]:
ydata = actions[200][:, 0]
xdata = np.arange(len(ydata))

In [ ]:
A, K = fit_exp_linear(xdata, ydata, C=0)
#fit_y = model_func(t, A, K, C0)
plot(ax2, t, y, noisy_y, fit_y, (A0, K0, C0), (A, K, 0))
ax2.set_title('Linear Fit')

In [ ]:
popt, pcov = curve_fit(exp_fn, xdata, ydata)

In [ ]:
fig, ax = plt.subplots()
ax.plot(xdata, exp_fn(xdata, *popt), ls='-',
        label='fit: a=%5.3f, b=%5.3f, c=%5.3f, d=%5.3f' % tuple(popt))
ax.plot(xdata, ydata, marker='o', ls='', label='Sample 0')
ax.set_xlabel('Step')
ax.set_ylabel('Total action')
plt.show()

## Look at observables from samples generated during training 

In [ ]:
log_dir = '../../gauge_logs_graph/run_10/'
train_observables_dicts = calc_training_observables(log_dir)

In [ ]:
train_observables_dicts = calc_training_observables(log_dir, 
                                                    train_observables_dicts)

In [ ]:
figs_axes = plot_training_observables(log_dir, train_observables_dicts)

In [ ]:
#actions_dict, plaqs_dict, charges_dict = train_observables_dicts
plot_top_charges_training(log_dir, train_observables_dicts[-1])

### OLD

In [ ]:
from utils.gauge_observables import (
    _load_samples_from_file, _load_params, _calc_observables,
)

In [ ]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 10000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 50,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'euc2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

save_params_to_pkl_file(params, info_dir)

In [ ]:
log_dir = '../../gauge_logs_graph/run_63/'
samples_dir = os.path.join(log_dir, 'samples_history/')
train_samples_dir = os.path.join(samples_dir, 'training/')
info_dir = os.path.join(log_dir, 'run_info/')

params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 1000,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 10000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 100,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'euc2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

save_params_to_pkl_file(params, info_dir)

In [ ]:
log_dir = '../../gauge_logs_graph/run_78/'
samples_dir = os.path.join(log_dir, 'samples_history/')
train_samples_dir = os.path.join(samples_dir, 'training/')
info_dir = os.path.join(log_dir, 'run_info/')

#save_params_to_pkl_file(params, info_dir)

params = _load_params(log_dir)

figs_dir = os.path.join(log_dir, 'figures')
samples_dict, actions_dict, plaqs_dict, charges_dict = {}, {}, {}, {}

In [ ]:
train_samples_files = [train_samples_dir + i 
                       for i in os.listdir(train_samples_dir) 
                       if i.endswith('.pkl')]
train_samples_files

In [ ]:
step_keys = sorted(
    [int(i.split('/')[-1].split('_')[2]) for i in train_samples_files]
)
step_keys

In [ ]:
training_figs_dir = os.path.join(figs_dir, 'training/')
check_else_make_dir(training_figs_dir)
training_steps_figs_dir = {}

for key in step_keys:
    _dir = os.path.join(training_figs_dir, f'{key}_train_steps/')
    check_else_make_dir(_dir)
    training_steps_figs_dir[key] = _dir

In [ ]:
for idx, sample_file in enumerate(train_samples_files):
    step = step_keys[idx]
    if step not in charges_dict.keys():
        print(f"Calculating observables for {step}...")
        with open(sample_file, 'rb') as f:
            samples = pickle.load(f)

        actions, plaqs, charges = _calc_observables(samples, params)

        actions_dict[step] = actions
        plaqs_dict[step] = plaqs
        charges_dict[step] = charges

In [ ]:
for key in charges_dict.keys():
    actions = actions_dict[key]
    plaqs = plaqs_dict[key]
    charges = charges_dict[key]
    observables = (actions, plaqs, charges)
    
    title_str = (r"$\beta =$"
                 + f"{params['beta_final']}, {key} training steps")
    
    kwargs = {
        'figs_dir': training_steps_figs_dir[key],
        'title': title_str
    }
    
    figs_axes = make_multiple_lines_plots(
        params['beta_final'],  
        observables, 
        **kwargs
    )

In [ ]:
plt.close('all')
for key, val in charges_dict.items():
    for idx in range(val.shape[1]):
        fig, ax = plt.subplots()
        _ = ax.plot(val[:, idx], 
                    marker=MARKERS[idx], color=COLORS[idx], 
                    ls='', fillstyle='none', label=f'sample {idx}')
        _ = ax.legend(loc='best')
        _ = ax.set_xlabel('Step', fontsize=14)
        _ = ax.set_ylabel('Topological charge', fontsize=14)
        title_str = (r"$\beta =$"
                     + f"{params['beta_final']}, {key} training steps")
        _ = ax.set_title(title_str, fontsize=16)
        out_file = os.path.join(training_steps_figs_dir[key],
                                f'topological_charge_history_sample_{idx}.pdf')
        if not os.path.isfile(out_file):
            print(f"Saving figure to: {out_file}.")
            _ = fig.savefig(out_file, dpi=400, bbox_inches='tight')
        
        #_ = ax.set_title(fr"""$\beta =$ {params['beta_final']},"""
        #                 fr""" {key} training steps""")
                         

## Modifying lattice structure

In [ ]:
from lattice.lattice import GaugeLattice

In [ ]:
lattice = GaugeLattice(8, 8, 2, 'U1', 5, False)

In [ ]:
lattice.links.shape

In [ ]:
sample = lattice.samples[0]

In [ ]:
sample

In [ ]:
lattice.sites

In [ ]:
links_arr = np.arange(lattice.num_links).reshape(sample.shape)
sites_arr = np.arange(lattice.num_sites).reshape(lattice.sites.shape)

In [ ]:
sites_arr = np.array(lattice.num_sites * ['o']).reshape(lattice.sites.shape)

In [ ]:
print(sites_arr)
print(sites_arr.shape)

In [ ]:
print(links_arr.T)
print(links_arr.T.shape)

In [ ]:
print(links_arr)

## OLD PLOTS

In [ ]:
ax1 = plt.subplot(311)
_ = plt.plot(top_charges[100][:, 0], label=f'sample 1', color='C0', ls=':')
             #markersize=2.5, marker=MARKERS[0], ls=':')
_ = plt.setp(ax1.get_xticklabels(), visible=False)

# share x only
_ = ax2 = plt.subplot(312, sharex=ax1)
_ = plt.plot(top_charges[100][:, 1], label=f'sample 2', color='C1', ls=':')
             #markersize=2.5, marker=MARKERS[1], ls=':')
_ = plt.setp(ax2.get_xticklabels(), visible=False)

_ = ax3 = plt.subplot(313, sharex=ax1)
_ = plt.plot(top_charges[100][:, 2], label=f'sample 3', color='C2', ls=':')
             #markersize=2.5, marker=MARKERS[2], ls=':')
#_ = plt.setp(ax3.get_xticklabels(), visible=False)

In [ ]:
MARKERS = ['o', 's', 'x', 'v', 'h', '^', 'p', '<', 'd', '>', 'o']
LINESTYLES = ['-', '--', ':', '-.', '-', '--', ':', '-.', '-', '--']

ax1 = plt.subplot(511)

_ = plt.plot(top_charges[100][:, 0], label=f'sample 1', color='C0', ls=':')
             #markersize=2.5, marker=MARKERS[0], ls=':')
_ = plt.setp(ax1.get_xticklabels(), visible=False)

# share x only
_ = ax2 = plt.subplot(512, sharex=ax1)
_ = plt.plot(top_charges[100][:, 1], label=f'sample 2', color='C1', ls=':')
             #markersize=2.5, marker=MARKERS[1], ls=':')
_ = plt.setp(ax2.get_xticklabels(), visible=False)

_ = ax3 = plt.subplot(513, sharex=ax1)
_ = plt.plot(top_charges[100][:, 2], label=f'sample 3', color='C2', ls=':')
             #markersize=2.5, marker=MARKERS[2], ls=':')
_ = plt.setp(ax3.get_xticklabels(), visible=False)

_ = ax4 = plt.subplot(514, sharex=ax1)
_ = plt.plot(top_charges[100][:, 3], label=f'sample 4', color='C3', ls=':')
             #markersize=2.5, marker=MARKERS[3], ls=':')
_ = plt.setp(ax4.get_xticklabels(), visible=False)

_ = ax5 = plt.subplot(515, sharex=ax1)
_ = plt.plot(top_charges[100][:, 4], label=f'sample 5', color='C4', ls=':')
             #markersize=2.5, marker=MARKERS[4], ls=':')
_ = plt.setp(ax5.get_xticklabels(), visible=False)

out_file = os.path.join(figs_dir_dict[100], 'top_charges_sharedx.pdf')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
plt.close('all')

In [ ]:
# Create plots for observables.
multiple_lines_figs_axes = make_multiple_lines_plots(
    figs_dir,
    params['beta'],
    observables,
    top_charges_autocorr,
    legend=False
)

# Create plots for observables with broken x-axes.
broken_xaxis_figs_axes = make_broken_xaxis_plots(
    figs_dir,
    params['beta'],
    observables,
    top_charges_autocorr,
    legend=False
)

# Plot lag k autocorrelation function of topological charge
# Plot topological charges autocorrelation function using the 
# built-in `pandas.plotting.autocorrelation_plot` method.
for idx in range(top_charges.shape[1]):
    out_file = os.path.join(
        pandas_autocorr_dir, 
        f'top_charges_autocorr_pandas_{idx}.pdf'
    )
    fig, ax = make_pandas_autocorrelation_plot(
        top_charges[:, idx],
        x_label='Lag',
        y_label='Autocorrelation (top. charge)',
        out_file=out_file
    )
    
# Plot topological charges autocorrelation function using the 
# built-in matplotlib `acorr` method.
for idx in range(top_charges.shape[1]):
    out_file = os.path.join(
        matplotlib_autocorr_dir, 
        f'top_charges_autocorr_matplotlib_{idx}.pdf'
    )
    kwargs = {'x_label': 'Lag',
              'y_label': 'Autocorrelation (top. charge)',
              'label': f'sample {idx}',
              'out_file': out_file,
              'color': COLORS[idx]}
    output = make_matplotlib_autocorrelation_plot(
        top_charges[:, idx],
        **kwargs
    )
    
# Compute and plot the samples autocorrelation spectrum.
# This is done by computing the autocorrelation function of each 
# individual link and then averaging over all links in the sample.
out_file = os.path.join(figs_dir, 'links_autocorrelation_vs_step.pdf')
fig, ax = make_samples_acl_spectrum_plot(samples, out_file=out_file)

    
# Compute the integrated autocorrelation time (IAT) 
# from top. charges data using `tau` from `utils/gauge_observables`
out_file = os.path.join(figs_dir, 'integrated_autocorrelation_time_plot.pdf')
kwargs = {
    'x_label': 'Lag',
    'y_label': 'Autocorrelation (top. charge)',
    'legend': True,
    'out_file': out_file
}
fig, ax = plot_autocorr_with_iat(acf_arr, iat_arr, ESS_arr, **kwargs)
#fig, ax = calc_integrated_autocorr_time_with_plots(top_charges, **kwargs)
#_ = ax.legend(bbox_to_anchor=(1, 0), loc="lower left",
#              bbox_transform=ax.transAxes, columnspacing=0.5, ncol=1)
#print(f"Saving figure to: {out_file}")
#plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
# Create plots for observables.

multiple_lines_figs_axes = make_multiple_lines_plots(
    figs_dir,
    params['beta'],
    observables,
    top_charges_autocorr,
    legend=False
)

# Create plots for observables with broken x-axes.
broken_xaxis_figs_axes = make_broken_xaxis_plots(
    figs_dir,
    params['beta'],
    observables,
    top_charges_autocorr,
    legend=False
)

# Plot lag k autocorrelation function of topological charge
# Plot topological charges autocorrelation function using the 
# built-in `pandas.plotting.autocorrelation_plot` method.
for idx in range(top_charges.shape[1]):
    out_file = os.path.join(
        pandas_autocorr_dir, 
        f'top_charges_autocorr_pandas_{idx}.pdf'
    )
    fig, ax = make_pandas_autocorrelation_plot(
        top_charges[:, idx],
        x_label='Lag',
        y_label='Autocorrelation (top. charge)',
        out_file=out_file
    )
    
# Plot topological charges autocorrelation function using the 
# built-in matplotlib `acorr` method.
for idx in range(top_charges.shape[1]):
    out_file = os.path.join(
        matplotlib_autocorr_dir, 
        f'top_charges_autocorr_matplotlib_{idx}.pdf'
    )
    kwargs = {'x_label': 'Lag',
              'y_label': 'Autocorrelation (top. charge)',
              'label': f'sample {idx}',
              'out_file': out_file,
              'color': COLORS[idx]}
    output = make_matplotlib_autocorrelation_plot(
        top_charges[:, idx],
        **kwargs
    )
    
# Compute and plot the samples autocorrelation spectrum.
# This is done by computing the autocorrelation function of each 
# individual link and then averaging over all links in the sample.
out_file = os.path.join(figs_dir, 'links_autocorrelation_vs_step.pdf')
fig, ax = make_samples_acl_spectrum_plot(samples, out_file=out_file)

# Compute the integrated autocorrelation time (IAT) 
# from top. charges data using `tau` from `utils/gauge_observables`
out_file = os.path.join(figs_dir, 'integrated_autocorrelation_time_plot.pdf')
kwargs = {
    'x_label': 'Lag',
    'y_label': 'Autocorrelation (top. charge)',
    'legend': True,
    'out_file': out_file
}
fig, ax = plot_autocorr_with_iat(acf_arr, iat_arr, ESS_arr, **kwargs)
#fig, ax = calc_integrated_autocorr_time_with_plots(top_charges, **kwargs)
#_ = ax.legend(bbox_to_anchor=(1, 0), loc="lower left",
#              bbox_transform=ax.transAxes, columnspacing=0.5, ncol=1)
#print(f"Saving figure to: {out_file}")
#plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
plt.close('all')

In [ ]:
out_file = os.path.join(figs_dir, 'integrated_autocorrelation_time_plot.pdf')
kwargs = {
    'x_label': 'Lag',
    'y_label': 'Autocorrelation (top. charge)',
    'legend': True,
    'out_file': out_file
}
fig, ax = plot_autocorr_with_iat(acf_arr, iat_arr, ESS_arr, **kwargs)

## OLD

In [ ]:
np.mean(ESS_arr)

In [ ]:
alphas = [0.3, 0.275, 0.25, 0.225, 0.2, 0.175, 
          0.15, 0.125, 0.1, 0.075, 0.05, 0.025][::-1]
out_file = os.path.join(matplotlib_autocorr_dir, 
                        'top_charges_autocorr_matplotlib.pdf')
fig, ax = plt.subplots()
for idx in range(top_charges.shape[1]):
    output = ax.acorr(top_charges[:, idx], usevlines=True, color=COLORS[idx],
                      normed=True, maxlags=None,
                      alpha=alphas[idx]+0.4, #zorder=zorders[idx],
                      label=f'sample {idx}')

_ = ax.axhline(0, color='r', lw=2)
_ = ax.grid(True)
_ = ax.legend(loc='best')
    
_ = ax.set_xlabel("Lag", fontsize=14)
_ = ax.set_ylabel("Autocorrelation (top. charge)", fontsize=14)
print(f'Saving figure to: {out_file}.')
_ = plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
lattice = GaugeLattice(8, 8, 2, 8., 'U1', 5, False)

In [ ]:
lattice.num_links

##### Old approach (unsure of validity)

In [ ]:
len_by_4 = len(top_charges) // 4
len_by_2 = len(top_charges) // 2
len_by_10 = len(top_charges) // 10
kappa4 = len(top_charges) - len_by_4
kappa2 = len(top_charges) - len_by_2
kappa10 = len(top_charges) - len_by_10
iac2, autocorr2 = calc_iat(top_charges.mean(axis=1), kappa=kappa2)
iac4, autocorr4 = calc_iat(top_charges.mean(axis=1), kappa=kappa4)
iac10, autocorr10 = calc_iat(top_charges.mean(axis=1), kappa=kappa10)

In [ ]:
iac2, iac4, iac10

In [ ]:
out_file = os.path.join(autocorr_dir, 
                        f'integrated_autocorrelation_time_plot_{kappa2}.pdf')

fig, ax = plt.subplots()
_ = ax.plot(np.arange(len(autocorr2)), autocorr2, ls='-')
_ = ax.set_xlabel('Lag', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (top. charge)', fontsize=14)
_ = ax.set_title(f'Integrated autocorrelation time (IAC): {iac2:6.4g}')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
out_file = os.path.join(figs_dir, 
                        f'integrated_autocorrelation_time_plot_{kappa4}.pdf')

fig, ax = plt.subplots()
_ = ax.plot(np.arange(len(autocorr4)), autocorr4, ls='-')
_ = ax.set_xlabel('Lag', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (top. charge)', fontsize=14)
_ = ax.set_title(f'Integrated autocorrelation time (IAC): {iac4:6.4g}')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
out_file = os.path.join(figs_dir, 
                        f'integrated_autocorrelation_time_plot_{kappa10}.pdf')

fig, ax = plt.subplots()
_ = ax.plot(np.arange(len(autocorr10)), autocorr10, ls='-')
_ = ax.set_xlabel('Lag', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (top. charge)', fontsize=14)
_ = ax.set_title(f'Integrated autocorrelation time (IAC): {iac10:6.4g}')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

## Old method (make each plot by hand)

### Specify run directory containing parameters and samples 

In [ ]:
log_dir = '../../gauge_logs_graph/run_2/'
info_dir = os.path.join(log_dir, 'run_info')
figs_dir = os.path.join(log_dir, 'figures')
params_file = os.path.join(info_dir, 'parameters.pkl')
with open(params_file, 'rb') as f:
    params = pickle.load(f)

In [ ]:
# Create lattice with same parameters to use for calculating observables
lattice = GaugeLattice(params['time_size'],
                       params['space_size'],
                       params['dim'],
                       params['beta'],
                       params['link_type'],
                       params['num_samples'],
                       params['rand'])

In [ ]:
# Load samples from `info_dir/samples_history.pkl` file
# Note that samples_history will be an array of shape:
#    [num_samples, num_eval_steps]
# where num_samples is the number of samples in each batch
# and num_eval steps is the number of steps the (trained) L2HMC simulation 
# was ran for.
samples_history_file = os.path.join(info_dir, 'samples_history.pkl')
with open(samples_history_file, 'rb') as f:
    samples_history = pickle.load(f)

In [ ]:
np.array(samples_history).shape

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
samples_history = np.array(samples_history)
actions_history = np.array(actions_history)
avg_plaquettes_history = np.array(avg_plaquettes_history)
top_charges_history = np.array(top_charges_history)
steps = np.arange(len(actions_history))

In [ ]:
len(samples_history[0].shape)

In [ ]:
# Compute the autocorrelation function using the topological charges
top_charges_autocorr_arr = []
num_samples = top_charges_history.shape[1]
for i in range(num_samples):
    top_charges_autocorr_arr.append(autocorr(top_charges_history[:, i]))
top_charges_autocorr_arr = np.array(top_charges_autocorr_arr)
top_charges_autocorr_avg = np.mean(top_charges_autocorr_arr, axis=0)

In [ ]:
top_charges_autocorr_arr.shape

In [ ]:
top_charges_autocorr_avg.shape

In [ ]:
samples_history = np.array(samples_history)
_shape = samples_history.shape
samples_history = samples_history.reshape(_shape[0], _shape[1], -1)
num_samples = samples_history.shape[1]
num_links  = samples_history.shape[-1]
samples_autocorr_arr = []
for n in range(num_samples):
    links_autocorr_arr = []
    for l in range(num_links):
        links_autocorr_arr.append(autocorr(samples_history[:, n, l]))
    samples_autocorr_arr.append(links_autocorr_arr)
samples_autocorr_arr = np.array(samples_autocorr_arr)
samples_autocorr_arr_avg = samples_autocorr_arr.mean(axis=1)

In [ ]:
samples_history.shape

In [ ]:
samples_autocorr_arr.shape

In [ ]:
samples_autocorr_arr_avg.shape

In [ ]:
out_file = os.path.join(figs_dir, 'topological_charge_autocorr_fn.pdf')
fig, ax = plot_multiple_lines(steps, top_charges_autocorr_arr,
                              x_label='step', 
                              y_label='Autocorrelation (top. charge)',
                              legend=True,
                              out_file=out_file)

In [ ]:
len(steps)

In [ ]:
out_file = os.path.join(figs_dir, 
                        'topological_charge_autocorr_fn_broken_xaxis.pdf')
fig, ax, ax2 = plot_broken_xaxis(steps, top_charges_autocorr_arr.T,
                                 xlabel='step',
                                 ylabel='Autocorrelation (top. charge)',
                                 #xlim1=(-2, 50), xlim2=(395, 500),
                                 output_file=out_file)

In [ ]:
out_file = os.path.join(figs_dir, 'topological_charge_vs_step.pdf')
fig, ax = plot_multiple_lines(steps, top_charges_history.T,
                              x_label='step', y_label='Topological charge',
                              out_file=out_file)

In [ ]:
out_file = os.path.join(figs_dir, 
                        'topological_charge_vs_step_broken_xaxis.pdf')
fig, ax, ax2 = plot_broken_xaxis(steps, top_charges_history,
                                 xlabel='step', ylabel='Topological charge',
                                 xlim1=(-2, 100), xlim2=(895, 1000),
                                 output_file=None)
ax2.legend(loc='lower right')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
from lattice.gauge_lattice import u1_plaq_exact

In [ ]:
out_file = os.path.join(figs_dir, 'average_plaquette_vs_step.pdf')
fig, ax = plot_multiple_lines(steps, avg_plaquettes_history.T,
                              x_label='step', y_label='Average plaquette')
_ = ax.axhline(y=u1_plaq_exact(params['beta']), 
           color='r', ls='--', lw=2.5, label='exact')
fig.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
out_file = os.path.join(figs_dir, 'average_plaquette_vs_step_broken_xaxis.pdf')
fig, ax, ax2 = plot_broken_xaxis(steps, avg_plaquettes_history,
                                 xlabel='step', ylabel='Average plaquette',
                                 xlim1=(-2, 65), xlim2=(895, 1000),
                                 output_file=None)

_ = ax.axhline(y=u1_plaq_exact(params['beta']), 
           color='r', ls='--', lw=2.5, label='exact')

_ = ax2.axhline(y=u1_plaq_exact(params['beta']), 
                color='r', ls='--', lw=2.5, label='exact')
leg = ax2.legend(loc='lower right', fontsize=10)

plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
out_file = os.path.join(figs_dir, 'average_action_vs_step.pdf')
fig, ax = plot_multiple_lines(steps, actions_history.T,
                              x_label='step', y_label='Average plaquette')
fig.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
out_file = os.path.join(figs_dir, 'average_action_vs_step_broken_xaxis.pdf')
fig, ax, ax2 = plot_broken_xaxis(steps, actions_history,
                                 xlabel='step', ylabel='Total action',
                                 xlim1=(-2, 55), xlim2=(895, 1000),
                                 output_file=out_file)

In [ ]:
samples_acl_spectrum = acl_spectrum(samples_history, scale=1)

In [ ]:
samples_acl_spectrum = acl_spectrum(samples_history, scale=1)
acl_steps = np.arange(len(samples_acl_spectrum))
fig, ax = plt.subplots()
ax.plot(acl_steps, samples_acl_spectrum/samples_acl_spectrum[0])
ax.set_xlabel('step', fontsize=14)
ax.set_ylabel('Autocorrelation (avg. over links)', fontsize=14)
plt.savefig(os.path.join(figs_dir, 'links_autocorrelation_vs_step.pdf'),
            dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
from utils.gauge_observables import *

In [ ]:
log_dir = '../../gauge_logs_graph/run_2/'

In [ ]:
params, samples, observables = calc_observables_from_log_dir(log_dir)

In [ ]:
actions, avg_plaquettes, top_charges = observables

beta = params['beta']
figs_dir = os.path.join(log_dir, 'figures')

In [ ]:
top_charges_autocorr, _ = calc_top_charges_autocorr(top_charges)
samples_autocorr, _ = calc_samples_autocorr(samples)

In [ ]:
from utils.gauge_observables import _make_plots

In [ ]:
multiple_lines_figs_axes, broken_xaxis_figs_axes = _make_plots(
    figs_dir, 
    beta, 
    samples, 
    observables,
    top_charges_autocorr, 
    samples_autocorr
)

In [ ]:
make_plots_from_log_dir(log_dir)

## Compare Sample Autocorrelation across runs

In [ ]:
_dir = '../../gauge_logs_graph/'
dirs = [
    os.path.join(_dir, i) for i in os.listdir(_dir) if i .startswith('run')
]

In [ ]:
samples_dict = {}
params_dict = {}
lattice_dict = {}
for d in dirs:
    key = d.split('/')[-1]
    info_dir = os.path.join(d, 'run_info')
    samples_file = os.path.join(info_dir, 'samples_history.pkl')
    parameters_file = os.path.join(info_dir, 'parameters.pkl')
    try:
        with open(samples_file, 'rb') as f:
            samples_dict[key] = pickle.load(f)
        with open(parameters_file, 'rb') as f:
            params_dict[key] = pickle.load(f)
        lattice_dict[key] = GaugeLattice(time_size=params['time_size'],
                                         space_size=params['space_size'],
                                         dim=params['dim'],
                                         beta=params['beta'],
                                         link_type=params['link_type'],
                                         num_samples=params['num_samples'],
                                         rand=params['rand'])
    except FileNotFoundError:
        continue

In [ ]:
samples_autocorr_dict = {}
samples_autocorr_avg_dict = {}
samples_acl_spectrum_dict = {}
for key, samples_history in samples_dict.items():
    samples_history = np.array(samples_history)
    _shape = samples_history.shape
    samples_history = samples_history.reshape(_shape[0], _shape[1], -1)
    num_samples = samples_history.shape[1]
    num_links  = samples_history.shape[-1]
    samples_autocorr_arr = []
    for n in range(num_samples):
        links_autocorr_arr = []
        for l in range(num_links):
            links_autocorr_arr.append(autocorr(samples_history[:, n, l]))
        samples_autocorr_arr.append(links_autocorr_arr)
    samples_autocorr_arr = np.array(samples_autocorr_arr)
    samples_autocorr_arr_avg = samples_autocorr_arr.mean(axis=1)
    samples_autocorr_dict[key] = samples_autocorr_arr
    samples_autocorr_avg_dict[key] = samples_autocorr_arr_avg
    samples_acl_spectrum_dict[key] = acl_spectrum(samples_history, scale=1)

In [ ]:
samples_autocorr_avg_dict['run_37'].mean(axis=0).shape

In [ ]:
fig, ax = plt.subplots()
for key, samples_autocorr_avg in samples_autocorr_avg_dict.items():
    autocorr_avg_over_samples = samples_autocorr_avg.mean(axis=0)
    steps = np.arange(len(autocorr_avg_over_samples))
    _ = ax.plot(steps, autocorr_avg_over_samples, label=f'{key}')
_ = ax.set_xlabel('step', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (avg. over links)', fontsize=14)
_ = ax.legend(loc='best')
_ = plt.show()

In [ ]:
fig, ax = plt.subplots()
for key, samples_acl_spectrum in samples_acl_spectrum_dict.items():
    acl_steps = np.arange(len(samples_acl_spectrum))
    _ = ax.plot(acl_steps, samples_acl_spectrum/samples_acl_spectrum[0], 
                label=f'{key}')
_ = ax.set_xlabel('step', fontsize=14)
_ = ax.set_ylabel('Autocorrelation (avg. over links)', fontsize=14)
_ = ax.legend(loc='best')
#plt.savefig(os.path.join(figs_dir, 'links_autocorrelation_vs_step.pdf'),
#            dpi=400, bbox_inches='tight')
plt.show()

### Incorrect

In [ ]:
out_file = os.path.join(figs_dir, 'samples_acl_spectrum.pdf')
fig, ax = plot_multiple_lines(x_data=steps, 
                              y_data=samples_autocorr_arr[0, 0:20],
                              x_label='step',
                              y_label='Autocorrelation (links)',
                              legend=False,
                              out_file=out_file)

In [ ]:
out_file = os.path.join(figs_dir, 'samples_acl_spectrum_avg.pdf')
fig, ax = plot_multiple_lines(x_data=steps,
                              y_data=samples_autocorr_arr_avg,
                              x_label='step',
                              y_label='Autocorrelation (avg. over links)',
                              semilogy=False,
                              legend=True,
                              out_file=out_file)

## OLD

In [ ]:
tf.InteractiveSession.close

In [ ]:
tf.Session().close()

In [ ]:
#sess = tf.InteractiveSession()

In [ ]:
tf.reset_default_graph()

In [ ]:
lattice = GaugeLattice(8, 8, 2, 8., 'U1', 2, rand=False)

In [ ]:
samples = tf.convert_to_tensor(lattice.samples, dtype=tf.float32)
potential_fn = lattice.get_energy_function(samples)

In [ ]:
dynamics = gde.GaugeDynamicsEager(
    lattice=lattice,
    num_steps=5,
    eps=0.1,
    minus_loglikelihood_fn=potential_fn,
    conv_net=True,
    hmc=False,
    eps_trainable=True
)

In [ ]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(50)

In [ ]:
# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

In [ ]:
training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

In [ ]:
# Run 20 epochs in which the training dataset is traversed, followed by the
# validation dataset.
for _ in range(20):
  # Initialize an iterator over the training dataset.
  sess.run(training_init_op)
  for _ in range(10):
    sess.run(next_element)

  # Initialize an iterator over the validation dataset.
  sess.run(validation_init_op)
  for _ in range(5):
    sess.run(next_element)

In [ ]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element)

sess.run(iterator.initializer)
print(sess.run(result))  # ==> "0"
print(sess.run(result))  # ==> "2"
print(sess.run(result))  # ==> "4"
print(sess.run(result))  # ==> "6"
print(sess.run(result))  # ==> "8"
try:
  sess.run(result)
except tf.errors.OutOfRangeError:
  print("End of dataset")  # ==> "End of dataset"

In [ ]:
samples_placeholder = tf.placeholder(samples.dtype, samples.shape)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(samples_placeholder).batch(2)

In [ ]:
dataset.output_shapes

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(dynamics.apply_transition(samples_placeholder))

In [ ]:
dataset.apply(dynamics.apply_transition)

In [ ]:
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

In [ ]:
sess.run(iterator.initializer, feed_dict={samples_placeholder: samples})

In [ ]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)    # Take a placeholder to create a dataset
iterator = dataset.make_initializable_iterator()      # Create an initializable iterator
next_element = iterator.get_next()

with tf.Session() as sess:
    # Initialize an iterator over a dataset with 10 elements using placeholder.
    sess.run(iterator.initializer, feed_dict={max_value: 10}) 

    for i in range(10):
        value = sess.run(next_element)
        print(f"{value} ", end=" ")    # 0 1 2 3 ... 9

In [ ]:
dataset = tf.data.Dataset.from_tensors(samples_placeholder)

In [ ]:
dataset

In [ ]:
dataset.apply(dynamics.apply_transition)
dataset = dataset.map(dynamics.apply_transition)

In [ ]:
iterator = dataset.make_initializable_iterator()

In [ ]:
sess.run(iterator.initializer, 
         feed_dict={samples_placeholder: lattice.samples})

In [ ]:
next_element = iterator.get_next()

In [ ]:
result = dynamics.apply_transition(next_element)

In [ ]:
sess.run(result)

In [ ]:
result = dynamics.apply_transition(next_element)

In [ ]:
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element)

In [ ]:
features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on `dataset`...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})